In [28]:
pip install cartopy

In [68]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 8.2 MB/s eta 0:00:00


In [96]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Cargar el archivo NetCDF
ruta = "TEMPO_NO2_L2_V03_20250916T214329Z_S012G07.nc"
ds = xr.open_dataset(ruta)

# Mostrar las variables disponibles
print(ds)




<xarray.Dataset> Size: 9kB
Dimensions:      (xtrack: 2048, mirror_step: 132)
Coordinates:
  * xtrack       (xtrack) int32 8kB 0 1 2 3 4 5 ... 2043 2044 2045 2046 2047
  * mirror_step  (mirror_step) int32 528B 798 799 800 801 ... 926 927 928 929
Data variables:
    *empty*
Attributes: (12/38)
    tio_commit:                       ea5e519d35940b9df5076444f27937f50402731f
    product_type:                     NO2
    processing_level:                 2
    processing_version:               3
    sdpc_version:                     TEMPO_SDPC_v4.6.1
    scan_num:                         12
    ...                               ...
    collection_shortname:             TEMPO_NO2_L2
    collection_version:               1
    keywords:                         EARTH SCIENCE>ATMOSPHERE>AIR QUALITY>NI...
    summary:                          Nitrogen dioxide Level 2 files provide ...
    coremetadata:                     \nGROUP                  = INVENTORYMET...
    history:                     

In [97]:
import h5netcdf

ruta = "TEMPO_NO2_L2_V03_20250916T214329Z_S012G07.nc"

with h5netcdf.File(ruta, 'r') as f:
    print("Grupos disponibles:")
    print(list(f.groups.keys()))


Grupos disponibles:
['product', 'geolocation', 'support_data', 'qa_statistics']


In [98]:
import xarray as xr

ruta = "TEMPO_NO2_L2_V03_20250916T214329Z_S012G07.nc"

# Abrir grupo con las concentraciones
ds_product = xr.open_dataset(ruta, group='product')
print(ds_product)


<xarray.Dataset> Size: 8MB
Dimensions:                                  (mirror_step: 132, xtrack: 2048)
Dimensions without coordinates: mirror_step, xtrack
Data variables:
    main_data_quality_flag                   (mirror_step, xtrack) float32 1MB ...
    vertical_column_troposphere              (mirror_step, xtrack) float64 2MB ...
    vertical_column_stratosphere             (mirror_step, xtrack) float64 2MB ...
    vertical_column_troposphere_uncertainty  (mirror_step, xtrack) float64 2MB ...


In [99]:
ds_geo = xr.open_dataset(ruta, group='geolocation')
print(ds_geo)


<xarray.Dataset> Size: 16MB
Dimensions:                 (mirror_step: 132, xtrack: 2048, corner: 4)
Coordinates:
    time                    (mirror_step) datetime64[ns] 1kB ...
    latitude                (mirror_step, xtrack) float32 1MB ...
    longitude               (mirror_step, xtrack) float32 1MB ...
Dimensions without coordinates: mirror_step, xtrack, corner
Data variables:
    latitude_bounds         (mirror_step, xtrack, corner) float32 4MB ...
    longitude_bounds        (mirror_step, xtrack, corner) float32 4MB ...
    solar_zenith_angle      (mirror_step, xtrack) float32 1MB ...
    solar_azimuth_angle     (mirror_step, xtrack) float32 1MB ...
    viewing_zenith_angle    (mirror_step, xtrack) float32 1MB ...
    viewing_azimuth_angle   (mirror_step, xtrack) float32 1MB ...
    relative_azimuth_angle  (mirror_step, xtrack) float32 1MB ...


In [100]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# === 1. Abrir grupos ===
ruta = "TEMPO_NO2_L2_V03_20250916T214329Z_S012G07.nc"

ds_product = xr.open_dataset(ruta, group='product')
ds_geo = xr.open_dataset(ruta, group='geolocation')

# === 2. Extraer variables ===
var = ds_product['vertical_column_troposphere'].values.flatten()
lat = ds_geo['latitude'].values.flatten()
lon = ds_geo['longitude'].values.flatten()
solar_zenith = ds_geo['solar_zenith_angle'].values.flatten()
solar_azimuth = ds_geo['solar_azimuth_angle'].values.flatten()
viewing_zenith = ds_geo['viewing_zenith_angle'].values.flatten()
relative_azimuth = ds_geo['relative_azimuth_angle'].values.flatten()
time_values = np.repeat(ds_geo['time'].values, ds_geo.dims['xtrack'])



/tmp/ipython-input-1203156622.py:19: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  time_values = np.repeat(ds_geo['time'].values, ds_geo.dims['xtrack'])


In [101]:
# Convertir a datetime en pandas
time_series = pd.to_datetime(time_values)

In [102]:
import numpy as np
import pandas as pd

# Crear una máscara de validez (sin NaN)
mask_valid = (~np.isnan(lat)) & (~np.isnan(lon)) & (~np.isnan(var))&(~np.isnan(solar_zenith))&(~np.isnan(solar_azimuth))&(~np.isnan(viewing_zenith))&(~np.isnan(relative_azimuth))

# Añadirlas al DataFrame (usando la misma máscara de validez que usaste para NO2)
df = pd.DataFrame({
    'lat': lat[mask_valid],
    'lon': lon[mask_valid],
    'NO2': var[mask_valid],
    'solar_zenith': solar_zenith[mask_valid],
    'solar_azimuth': solar_azimuth[mask_valid],
    'viewing_zenith': viewing_zenith[mask_valid],
    'relative_azimuth': relative_azimuth[mask_valid],
    'time': time_series[mask_valid]
})

print(df.shape)
df.head()


(267867, 8)


,lat,lon,NO2,solar_zenith,solar_azimuth,viewing_zenith,relative_azimuth,time
0,58.103729,-112.039688,2.861060e+15,62.303814,-139.351654,68.500305,-114.926056,2025-09-16 21:43:47.410193664
1,58.063507,-112.011696,6.070829e+14,62.282948,-139.308243,68.455025,-114.904617,2025-09-16 21:43:47.410193664
2,58.029163,-111.988800,3.506375e+14,62.264816,-139.272217,68.416573,-114.886230,2025-09-16 21:43:47.410193664
3,57.993637,-111.964943,9.943956e+14,62.246151,-139.234741,68.376755,-114.867203,2025-09-16 21:43:47.410193664
4,57.954674,-111.938202,1.235008e+15,62.225918,-139.193008,68.332947,-114.846344,2025-09-16 21:43:47.410193664


In [103]:
df['hour'] = df['time'].dt.hour           # hora del día (0–23)
df['dayofyear'] = df['time'].dt.dayofyear # día del año (1–365)
df['month'] = df['time'].dt.month         # mes
df['weekday'] = df['time'].dt.weekday     # día de la semana (0–6)

In [104]:
import requests
import pandas as pd

# Ejemplo: coordenadas promedio de tu zona
lat = df['lat'].mean()
lon = df['lon'].mean()

# Solicitud a la API (predicción histórica)
url = (
    f"https://archive-api.open-meteo.com/v1/archive?"
    f"latitude={lat}&longitude={lon}"
    f"&start_date=2025-09-16&end_date=2025-09-16"
    f"&hourly=temperature_2m,pressure_msl,windspeed_10m,winddirection_10m"
)

r = requests.get(url)
data = r.json()

# Convertir a DataFrame
meteo = pd.DataFrame(data['hourly'])
meteo['time'] = pd.to_datetime(meteo['time'])
meteo.head()


,time,temperature_2m,pressure_msl,windspeed_10m,winddirection_10m
0,2025-09-16 00:00:00,21.4,1015.3,10.3,295
1,2025-09-16 01:00:00,18.2,1015.3,6.5,273
2,2025-09-16 02:00:00,15.0,1015.9,11.2,338
3,2025-09-16 03:00:00,14.6,1016.3,9.3,345
4,2025-09-16 04:00:00,13.4,1017.2,1.3,16


In [105]:
meteo.columns

Index(['time', 'temperature_2m', 'pressure_msl', 'windspeed_10m',
       'winddirection_10m'],
      dtype='object')

In [106]:
# Redondear al inicio de la hora
df['time_hour'] = df['time'].dt.floor('H')
meteo['time_hour'] = pd.to_datetime(meteo['time'])

/tmp/ipython-input-2203159017.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['time_hour'] = df['time'].dt.floor('H')


In [107]:
df = df.merge(
    meteo[['time_hour', 'temperature_2m', 'pressure_msl', 'windspeed_10m', 'winddirection_10m']],
    on='time_hour',
    how='left'
)


In [108]:
# Ordenar los datos por ubicación y tiempo
df = df.sort_values(by=['lat', 'lon', 'time'])

# Calcular el valor anterior de NO2 para cada pixel
df['NO2_prev'] = df.groupby(['lat', 'lon'])['NO2'].shift(1)



In [110]:
# Guardar en formato CSV
df.to_csv("TEMPO_NO2_L2_datos.csv", index=False, sep=";")

print("Archivo guardado: TEMPO_NO2_L2_datos.csv")

Archivo guardado: TEMPO_NO2_L2_datos.csv


In [109]:
df.head()

,lat,lon,NO2,solar_zenith,solar_azimuth,viewing_zenith,relative_azimuth,time,hour,dayofyear,month,weekday,time_hour,temperature_2m,pressure_msl,windspeed_10m,winddirection_10m,NO2_prev
4067,17.368040,-101.597099,8.667150e+14,47.332428,-103.401131,23.732944,-71.189430,2025-09-16 21:43:50.443265792,21,259,9,1,2025-09-16 21:00:00,23.0,1016.5,15.5,252,NaN
2035,17.368172,-101.553032,1.456828e+15,47.361755,-103.385391,23.707085,-71.283676,2025-09-16 21:43:47.410193664,21,259,9,1,2025-09-16 21:00:00,23.0,1016.5,15.5,252,NaN
6103,17.368628,-101.642288,9.511441e+14,47.302223,-103.418106,23.760258,-71.094910,2025-09-16 21:43:53.476332544,21,259,9,1,2025-09-16 21:00:00,23.0,1016.5,15.5,252,NaN
8133,17.369335,-101.687943,-9.738525e+12,47.271618,-103.435440,23.788050,-71.000015,2025-09-16 21:43:56.509402624,21,259,9,1,2025-09-16 21:00:00,23.0,1016.5,15.5,252,NaN
10168,17.369675,-101.732521,5.194994e+14,47.241924,-103.451912,23.814919,-70.906624,2025-09-16 21:43:59.542482688,21,259,9,1,2025-09-16 21:00:00,23.0,1016.5,15.5,252,NaN


In [89]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Tomar una muestra aleatoria
df_sample = df.sample(frac=1, random_state=42)

features = [
    'lat', 'lon',
    'solar_zenith', 'solar_azimuth',
    'viewing_zenith', 'relative_azimuth',
    'hour', 'dayofyear','temperature_2m', 'pressure_msl', 'windspeed_10m', 'winddirection_10m'
]

X = df_sample[features]
y = df_sample['NO2']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [90]:
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, KFold

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 600),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.5, 3),
        "n_jobs": -1,
        "random_state": 42
    }

    model = XGBRegressor(**params)
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, cv=kf, scoring="r2").mean()
    return score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1) #colocar 50 trials


[I 2025-10-02 23:06:52,473] A new study created in memory with name: no-name-518ec6e9-a1cc-41c2-a6d1-8a9885e8c726
[I 2025-10-02 23:07:24,045] Trial 0 finished with value: 0.07003951962899795 and parameters: {'n_estimators': 297, 'max_depth': 15, 'learning_rate': 0.2825014880868819, 'subsample': 0.9828666918939795, 'colsample_bytree': 0.761140322217322, 'min_child_weight': 10, 'gamma': 4.262928769467721, 'reg_alpha': 0.3325300165102617, 'reg_lambda': 0.9526965837083943}. Best is trial 0 with value: 0.07003951962899795.


In [91]:
from sklearn.metrics import r2_score

best_params = study.best_params
print("Mejores hiperparámetros:", best_params)

# Entrenar modelo final con todos los datos
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

# Evaluar en el conjunto de prueba
y_pred = best_model.predict(X_test)
print("R²:", r2_score(y_test, y_pred))


Mejores hiperparámetros: {'n_estimators': 297, 'max_depth': 15, 'learning_rate': 0.2825014880868819, 'subsample': 0.9828666918939795, 'colsample_bytree': 0.761140322217322, 'min_child_weight': 10, 'gamma': 4.262928769467721, 'reg_alpha': 0.3325300165102617, 'reg_lambda': 0.9526965837083943}
R²: 0.10702359749236667
